In [2]:
import sys
sys.path.append('/users/kuba/code/aic/mil')

import mil_pytorch.mil as mil
from mil_pytorch.utils import eval_utils, data_utils, train_utils, create_bags_simple


import numpy
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler
from sklearn.datasets import make_classification
import time



def create_model(n_neurons1, n_neurons2, n_neurons3):
    # Define neural networks for processing of data before and after aggregation
    prepNN1 = torch.nn.Sequential(
        torch.nn.Linear(len(dataset.data[0]), n_neurons1, bias = True),
        torch.nn.ReLU(),
        torch.nn.Linear(n_neurons1, n_neurons2, bias = True),
        torch.nn.ReLU(),
    )

    afterNN1 = torch.nn.Sequential(
        torch.nn.Identity()
    )

    prepNN2 = torch.nn.Sequential(
        torch.nn.Linear(n_neurons2, n_neurons3, bias = True),
        torch.nn.ReLU(),
    )

    afterNN2 = torch.nn.Sequential(
        torch.nn.Linear(n_neurons3, 1),
        torch.nn.Tanh()
    )

    # Define model ,loss function and optimizer
    model = torch.nn.Sequential(
        mil.BagModel(prepNN1, afterNN1, torch.mean, device),
        mil.BagModel(prepNN2, afterNN2, torch.mean, device)
    ).double()
    criterion = mil.MyHingeLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate, weight_decay = weight_decay)

    return model

# --- CONFIG ---

# Configurations
n_neurons1 = None
n_neurons2 = None
n_neurons3 = None
learning_rate = 1e-3
weight_decay = 1e-3
epochs = 4000
pos = 50
neg = 50
class_sep = 1.2
n_features = 100
max_subbags = 5
max_instances = 5
batch_size = 0
patience = 20
delta = 0

config = {}
config['epochs'] = epochs
config['pos'] = pos
config['neg'] = neg
config['class_sep'] = class_sep
config['n_features'] = n_features
config['max_subbags'] = max_subbags
config['max_instances'] = max_instances
config['batch_size'] = batch_size
config['patience'] = patience
config['delta'] = delta
config["n_neurons1"] = n_neurons1
config["n_neurons2"] = n_neurons2
config["n_neurons3"] = n_neurons3
config["learning_rate"] = learning_rate
config["weight_decay"] = weight_decay

# --- DATA ---

# Create data
source_data, source_labels = make_classification(n_samples = 20000, n_features = n_features, n_informative = n_features, n_redundant = 0, n_repeated = 0, n_classes = 10, class_sep = class_sep, n_clusters_per_class = 1)
data, ids, labels = create_bags_simple.create_bags(source_data, source_labels, pos = pos, neg = neg, max_instances = max_instances)
print("Data shape:", data.shape, len(labels))

# Check if gpu available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("INFO: Using device: {}".format(device))

# Move data to gpu (if available)
data = data.double().to(device)
ids = ids.long().to(device)
labels = labels.long().to(device)

# Convert labels from (1, 0) to (1, -1) for tanh
labels[labels == 0] = -1

# Create dataset and divide to train and test part
dataset = mil.MilDataset(data, ids, labels, normalize = True)

train_indices, valid_indices, test_indices = data_utils.data_split(dataset = dataset, valid_ratio = 0.0, test_ratio = 0.2, shuffle = True, stratify = True)

print('DEBUG: train: {}, valid: {}, test: {}'.format(train_indices, valid_indices, test_indices))

train_sampler = SubsetRandomSampler(train_indices)
# valid_sampler = SubsetRandomSampler(valid_indices)
test_sampler = SubsetRandomSampler(test_indices)

# Create dataloaders
if batch_size == 0:
    train_dl = DataLoader(dataset, sampler = train_sampler, batch_size = len(train_indices), collate_fn=mil.collate)
else:
    train_dl = DataLoader(dataset, sampler = train_sampler, batch_size = batch_size, collate_fn=mil.collate)

# valid_dl = DataLoader(dataset, sampler = valid_sampler, batch_size = len(valid_indices), collate_fn=mil.collate)
test_dl = DataLoader(dataset, sampler = test_sampler, batch_size = len(test_indices), collate_fn=mil.collate)


# ---- GRID SEARCH ---

n_neurons1_grid = [5, 10, 15]
n_neurons2_grid = [5, 10, 15]
n_neurons3_grid = [5, 10, 15]
learning_rate_grid = numpy.logspace(-4, -2, num = 3)
weight_decay_grid = numpy.logspace(-4, -2, num = 3)

for n_neurons1 in n_neurons1_grid:
    for n_neurons2 in n_neurons2_grid:
        for n_neurons3 in n_neurons3_grid:
            for learning_rate in learning_rate_grid:
                for weight_decay in weight_decay_grid:
                    config['n_neurons1'] = n_neurons1
                    config['n_neurons2'] = n_neurons2
                    config['n_neurons3'] = n_neurons3
                    config['learning_rate'] = learning_rate.item()
                    config['weight_decay'] = weight_decay.item()

                    print('INFO: Running cross validation with config:\n{}'.format(config))
 
                    # --- MODEL ---
                    model = create_model(n_neurons1, n_neurons2, n_neurons3)
                    criterion = mil.MyHingeLoss()
                    optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate, weight_decay = weight_decay)

                    # Move model to gpu if available
                    model = model.to(device)

                    avg_loss = train_utils.k_fold_cv(model = model, fit_fn = train_utils.train_model, criterion = criterion, optimizer = optimizer, dataset = dataset, train_indices = train_indices, epochs = epochs, patience = patience, delta = delta, device = device)

                    # Save log
                    train_utils.save_log(avg_loss, config)


Data shape: torch.Size([361, 100]) 100
INFO: Using device: cpu
DEBUG: train: [78 49 66 50 69 14 94 63 12 25 26  6 97 88 80 34 86 92 64 36 81 20 28 44
  5 52 31 57 85 43 16  2 22 95 72 24 45  4 32  3 96 58 76 89 75 23  7 65
 74 71 55 18 15 21 10 40 83 70 79 90 56 59 47 42 30 61 84 17 33 19 87  0
 37 11 73 60 99 82 62 53], valid: [], test: [39 54 13 27  1 38 67 98 77 48 93 41 35 51  8 29 68 46  9 91]
INFO: Running cross validation with config:
{'epochs': 4000, 'pos': 50, 'neg': 50, 'class_sep': 1.2, 'n_features': 100, 'max_subbags': 5, 'max_instances': 5, 'batch_size': 0, 'patience': 20, 'delta': 0, 'n_neurons1': 5, 'n_neurons2': 5, 'n_neurons3': 5, 'learning_rate': 0.0001, 'weight_decay': 0.0001}
K-fold CV: [1/5]
TRAINING:
DEBUG: shape of data: torch.Size([236, 100]) shape of info: torch.Size([1, 236]) shape of labels: torch.Size([64])
DEBUG: bags: tensor([65, 52, 62, 47, 79, 23, 99, 60, 75, 55, 19,  5, 30, 95,  0, 96, 40, 42,
        33, 15, 20, 57, 70, 82, 83, 76, 45, 90, 53, 32,  2, 

TypeError: len() of a 0-d tensor

In [3]:
dataset.ids

tensor([[ 0,  0,  0,  1,  1,  1,  1,  1,  2,  2,  3,  3,  3,  3,  4,  4,  5,  5,
          5,  5,  5,  6,  6,  6,  6,  6,  7,  7,  7,  7,  7,  8,  8,  8,  9,  9,
         10, 10, 11, 11, 11, 12, 12, 12, 12, 13, 13, 13, 13, 14, 14, 14, 15, 15,
         15, 15, 16, 16, 16, 17, 17, 17, 17, 18, 18, 18, 18, 18, 19, 19, 19, 19,
         20, 20, 20, 20, 21, 21, 22, 22, 22, 22, 23, 23, 23, 23, 23, 24, 24, 24,
         24, 25, 25, 25, 25, 25, 26, 26, 26, 26, 27, 27, 28, 28, 28, 29, 29, 29,
         29, 30, 30, 31, 31, 31, 31, 32, 32, 32, 33, 33, 33, 33, 34, 34, 34, 34,
         35, 35, 36, 36, 36, 37, 37, 38, 38, 38, 39, 39, 39, 40, 40, 40, 41, 41,
         42, 42, 42, 42, 43, 43, 43, 44, 44, 44, 44, 44, 45, 45, 45, 46, 46, 46,
         46, 47, 47, 47, 47, 47, 48, 48, 48, 49, 49, 49, 49, 49, 50, 50, 50, 50,
         50, 51, 51, 52, 52, 52, 52, 53, 53, 53, 53, 53, 54, 54, 54, 54, 54, 55,
         55, 56, 56, 57, 57, 57, 57, 57, 58, 58, 58, 58, 58, 59, 59, 59, 60, 60,
         60, 61, 61, 61, 61,